<a href="https://colab.research.google.com/github/koliby777/pokus-cislo/blob/master/Jazykovy_model_2_kytice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Pokusy s modelem pro predikci následujícího znaku pomocí transformerů***


**Co je vlastně ChatGPT?**

GPT = Generative Pre-training Transformer (Generativní předtréninkový transformer),

LLM = Large Language Model (velký jazykový model),

ChatGPT = GPT vyladěný pro povídání si s "umělou" inteligencí.

Dlaší systémy AI jsou např.: Llama, Claude, Gemini, Mistral, DALL-E ...

Obecně : https://drive.google.com/file/d/1pxx_ZI7O-Nwl7ZLNk5hI3WzAsTLwvNU7/view?usp=sharing


NN = Neural Network (neuronová síť): https://github.com/koliby777/pokus-cislo/blob/master/EU/neuron.png
  https://www.youtube.com/watch?v=IHZwWFHWa-w  

Neuronová síť: vstup (otázka/prompt) a výstup (odpověď modelu)

NN obecně řečeno vyjadřuje 2 druhy funkci:

- fce tranformování vstupu sítě ve výstup sítě (struktura sítě a parametry jedn. neuronů)

- fce chyby na výstupu v závislosti na parametrech všech neuronů

Výstup je funkcí vstupu  (transformační fce, stochastická)

K tomu NN obsahuje pro každý neuron tyto parametry:

  - jeho vstupy z předch. vrstvy neuronů

  - jeho váhy na vstupu do neuronu

  - jeho práh (bias) ("zkreslení")

  - jeho aktivační funkci (např. sigmoida)

  - jeho výstup z neuronu do další vrstvy neuronů
Hodnoty vstupů a výstupů z neuronů mají často chrakter pravděpodobnosti (0, 1).


Učení modelu (Pre-training):

1. parametry všech neuronů se náhodně vygenerují.

3. Na přípravu dat pro učení se používá tokenizer. Tokenizer je nástroj zpracování přirozeného jazyka, který rozděluje vstupní text na menší části, nazývané tokeny. Tyto tokeny mohou být slova, fráze, znaky nebo jiné jednotky, na základě kterých se provádí další analýza nebo zpracování textu, včetně kódování na čísla.

2. Na vstup celé sítě se pak dávají zakódovaná trénovací data a data validační, na výstupu se srovnávají odezvy sítě, vzniká výstupní chyba (nesoulad v chování modelu mezi trénovacími a validačními daty, training loss vs. validation loss). Chyba je tak velmi složitou funkcí parametrů všech neuronů v mnoharozměrném prostoru.

3. Pomocí algoritmu zpětného šíření chyby (backpropagation) se zpětně přepočítávají všechny paramatry modelu až do vstupní vrstvy modelu. Cílem je snížit výstupní chybu. Gradientní sestup vede ke snižování chyby, ideálně k nalezení jejího minima (aspoň s určitou tolerancí).

4. To celé (2. až 3.) se např. 10000 krát iteračně opakuje, až je nalezeno jisté minimum chyby. Výsledkem je pak množina parametrů všech neuronů sítě, která zajišťuje přijatelné chování modelu (otázka - odpověď). Odpovědi modelu se dekódují na čitelný text.

Používání modelu:
- zadání údajů o generování výstupů (počet znaků dílčí dávky odpovědi a počet iterací generování odpovědi)
- zadávání otázek (promptů) a získávání odpovědi modelu
- po skončení se poslední odpověď modelu přehraje jako řeč



******

3. Kytice
zdroj https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/kytice.txt
výstup z modelu  https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/10x%20KYTICE/10x%20kytice%20vystup%20PC.txt

4. Rozmarné léto
zdroj https://raw.githubusercontent.com/koliby777/pokus-cislo/master/ROZMARNE%20LETO/rozmarne%20leto.txt
výstup z modelu  https://raw.githubusercontent.com/koliby777/pokus-cislo/master/ROZMARNE%20LETO/vystup.txt

5. Sborník EU Paralelní korpus 1996 - 2011. https://www.statmt.org/europarl/ European Parliament Proceedings Parallel Corpus 1996-2011. Použity texty v češtině a angličtině 183 MB: 13 mil. slov českých v 669 tis. větách. A 16 mil. slov anglických.
Výstup z modelu  https://raw.githubusercontent.com/koliby777/pokus-cislo/master/EU/eu%20trenovani%20%20best.txt






4. Parametry natrénovaného modelu EU:

  Učení trvalo cca 2,5 hodiny, na notebooku Eurocom s GPU NVIDIA GeForce RTX 2080 (Cuda)

  Počet parametrů modelu je přes 23 miliónů, které zabírají 108 MB (původní texty EU jsou 183 MB).

  To není komprese dat jako např. "zazipování", ale o zaznamenání urč. pravděpodobností výskytu slov apod., t.j. model si nepamatuje přesně doslova, co se naučil - zde je podobnost s lidským mozkem (!).

  Na zadanou otázku v ČJ či AJ, model odpoví "co ho napadne", a to v jazyce otázky (téměř vždy).
  
  Na stejnou otázku zpravidla dává různé, i když tématicky podobné odpovědi.

  Ovšem pro lepší fungování by model musel být podstaně lépe učen: síť by musela být podstatně větší, počet iterací učení rovněž.



4. Praktická ukázka  Kytice a EU








In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from google.colab import drive
!pip install gtts langdetect
from gtts import gTTS
from langdetect import detect
from IPython.display import Audio, display
import tempfile
import textwrap
!pip install gdown
import gdown
print("\nGPU CUDA je k dispozici: ", torch.cuda.is_available())

In [ ]:
# hyperparametry
batch_size = 64 # určuje, kolik nezávislých sekvencí bude zpracováváno paralelně
block_size = 256 # maximální délka kontextu pro predikce
max_iters =   10000 # maximální počet iterací trénování
eval_interval = 500 # interval pro evaluaci modelu
learning_rate = 3e-4 # rychlost učení
device = 'cuda' if torch.cuda.is_available() else 'cpu' # zařízení pro výpočty, GPU pokud je dostupné, jinak CPU
eval_iters = 200 # počet iterací pro vyhodnocení
n_embd = 384 # velikost vektorů vložení
n_head = 6 # počet hlav v "multi-head attention"

n_layer = 11 # počet vrstev transformeru   !!!!!!!! kytice 11, EU 13  !!!!!

dropout = 0.2 # pravděpodobnost zapomínání naučeného


torch.manual_seed(1337) # nastaví náhodný seed pro reprodukovatelnost

"""
# ad EU
!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/EU/eu!!!.txt
with open('eu!!!.txt', 'r', encoding='utf-8') as f:
    text = f.read() # načte textový soubor k učení
"""
# ad Kytice
!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/10x%20KYTICE/100x%20kytice.txt
with open('100x kytice.txt', 'r', encoding='utf-8') as f:
    text = f.read() # načte textový soubor k učení


# zde jsou všechny unikátní znaky, které se v textu vyskytují
chars = sorted(list(set(text))) # vytvoří seznam unikátních znaků
vocab_size = len(chars) # počet unikátních znaků
print("----------------------------------------------------")
znaky = ''.join(chars) # spojení všech unikátních znaků do jednoho řetězce bez jakýchkoli oddělovačů
zalamovany_text = textwrap.fill(znaky, width=50)
print(f"Model používá těchto {vocab_size} znaků:\n {zalamovany_text}") # tisk seznamu unikátních znaků
# print(znaky)
# print(f"{len(znaky)} znaků")

# vytvoří mapování znaků na celá čísla
stoi = { ch:i for i,ch in enumerate(chars) } # mapování znak na index
itos = { i:ch for i,ch in enumerate(chars) } # mapování index na znak
encode = lambda s: [stoi[c] for c in s] # encoder: převede řetězec na seznam čísel
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: převede seznam čísel zpět na řetězec

In [ ]:
# Samotný model

# Rozdělení dat na trénovací a validační
data = torch.tensor(encode(text), dtype=torch.long) # zakóduje celý text do tensoru
n = int(0.9*len(data)) # prvních 90% dat bude trénovacích, zbytek validačních
train_data = data[:n] # trénovací data
val_data = data[n:] # validační data

# načítání dat
def get_batch(split):
    # vygeneruje malý batch dat pro vstupy x a cíle y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # náhodně vybere začátky sekvencí
    x = torch.stack([data[i:i+block_size] for i in ix]) # vytvoří vstupy x
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # vytvoří cíle y (o jednu pozici posunuté)
    x, y = x.to(device), y.to(device) # přesune data na zvolené zařízení
    return x, y

@torch.no_grad() # deaktivuje výpočet gradientů pro zvýšení efektivity
def estimate_loss():
    out = {}
    model.eval() # přepne model do evaluačního módu
    for split in ['train', 'val']: # pro trénovací a validační data
        losses = torch.zeros(eval_iters) # inicializuje tensor pro ukládání ztrát
        for k in range(eval_iters): # pro každou evaluační iteraci
            X, Y = get_batch(split) # získá dávku dat
            logits, loss = model(X, Y) # vypočítá logity a ztrátu
            losses[k] = loss.item() # uloží hodnotu ztráty
        out[split] = losses.mean() # průměrná ztráta pro daný split
    model.train() # přepne model zpět do trénovacího módu
    return out

class Head(nn.Module): # definice jedné hlavy self-attention
    """ jedna hlava self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)

        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):


    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):


    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):


    def __init__(self, n_embd, n_head):

        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# jednoduchý bigramový model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape


        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):

        for _ in range(max_new_tokens):

            idx_cond = idx[:, -block_size:]

            logits, loss = self(idx_cond)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)

            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

print("----------------------------------------------------")
print("\nModel obsahuje ", sum(p.numel() for p in m.parameters())/1e6, 'miliónů parametrů celkem!\n')
# ma byt  23.368986  miliónů parametrů celkem u EU
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
print("Optimalizátor vytvořen.")


In [ ]:
# učení modelu:
"""
for iter in range(max_iters):


    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("naučeno !!!!!\n")
"""

In [ ]:
# připojení disku Google:
# drive.mount('/content/drive')

In [ ]:
# uložení parametrů modelu na disk Google:
"""
path = '/content/drive/My Drive/MyModels/eu.pth'
torch.save(m, path)
print("\nModel byl uložen\n")
"""

In [ ]:
# načtení parametrů modelu z disku Google:
"""
#ad EU
path = '/content/drive/My Drive/MyModels/cs-en.pth'
m = torch.load(path, map_location=torch.device(device))
print("\nModel byl načten.\n")
"""

# ad Kytice
"""
path = '/content/drive/My Drive/MyModels/100x kytice.pth'
m = torch.load(path, map_location=torch.device(device))
print("\nModel byl načten.\n")
"""

url = 'https://drive.google.com/uc?id=1scImJYF2hy6J2PVpPvsN92NpkdwRpkDs'
output = '100x kytice.pth'

# Stažení souboru z Google Drive
gdown.download(url, output, quiet=False)

# Načtení modelu
m = torch.load(output, map_location=torch.device(device))


In [ ]:

# používání modelu:
def dotaz():
    print("----------------------------------------------------")
    print('Zadej dotaz: (KONEC pro skočení)')
    text = input()
    return text

def zadej_cislo():
  t_f = True
  chyba = "to není přirozené číslo!"
  while t_f:
    try:
      cislo = int(input())
      if cislo < 1:
        print(chyba)
      else:
        return cislo
        t_f = False
    except ValueError:
      print(chyba)

# Nastavení šířky zalomení textu

print("----------------------------------------------------")
print('Zadej počet znaků dílčí dávky odpovědi modelu: (10 např.)')
partial_batch = zadej_cislo()
print("----------------------------------------------------")
print('Zadej počet iterací generování odpovědi: (30 např.)')
max = zadej_cislo()

vstup = ""
while vstup != "KONEC":
    vystup = ""
    initial_text = dotaz()
    vstup = initial_text
    if vstup == "KONEC":
      break
    ii = 1
    while ii <= max:
        initial_tokens = encode(initial_text)
        context_tensor = torch.tensor([initial_tokens], dtype=torch.long).to(device)  # Přidáváme dimenzi pro dávku
        generated_tokens = m.generate(context_tensor, max_new_tokens=partial_batch)  # Generuje zadaný počet tokenů
        generated_text = decode(generated_tokens[0, -partial_batch:].tolist())
        vystup = vystup + generated_text
        initial_text = generated_text
        ii += 1

    print("----------------------------------------------------")
    # print(f"Výstup: \n {vystup}")
    # Použití funkce fill() z modulu textwrap pro zalomení textu

    """
    # pro EU
    vystup = vystup.replace("!", " ")
    zalamovany_text = textwrap.fill(vystup, width = 70)
    """

    zalamovany_text = vystup # pro Kytici


    # Tisk zalomeného textu
    print(f"Výstup: \n{zalamovany_text}")

lang = detect(zalamovany_text)
print(f"\nPoslední odezva modelu je v jazyce {lang}:\n")
tts = gTTS(text=zalamovany_text, lang=lang)
with tempfile.NamedTemporaryFile(delete=True, suffix='.mp3') as tmpfile:
        tts.save(tmpfile.name)
        display(Audio(tmpfile.name, autoplay=True))


print("...skončeno.")